<a href="https://colab.research.google.com/github/Maxi1324/TetrisAi2GoogleColab/blob/main/AINormal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pillow
!pip install numpy
!pip install pygame
!pip install keras-rl2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np 

import PIL.Image as Image
import gym
import random
from gym import Env, spaces
import time
import pygame
from pygame import gfxdraw

class TetrisBlock:
    blockColors = [
        (255, 0, 0), # 1 = red
        (0, 255, 0), # 2 = green
        (0, 0, 255), # 3 = blue
        (255, 255, 0), # 4 = yellow
        (255, 0, 255), # 5 = magenta
        (0, 255, 255), # 6 = cyan
        (255, 255, 255), # 7 = white
        (255, 127, 0), # 8 = orange
    ]
    blocktypes = [
      [[0,1,0,0],
       [0,1,0,0],
       [0,1,0,0],
       [0,1,0,0]],

      [[0,0,0,0],
        [0,1,1,0],
        [0,1,1,0],
        [0,0,0,0]],

      [[0,0,0,0],
        [0,1,0,0],
        [0,1,1,0],
        [0,0,1,0]],

      [[0,0,0,0],
        [0,0,1,0],
        [0,1,1,0],
        [0,1,0,0]],

      [[0,0,0,0],
        [0,1,1,0],
        [0,0,1,0],
        [0,0,1,0]],

      [[0,0,0,0],
        [0,0,1,0],
        [0,1,1,0],
        [0,0,1,0]],

      [[0,0,0,0],
        [0,1,1,0],
        [0,1,0,0],
        [0,1,0,0]]
    ]

    rotateN = 0

    def __init__(self, blockType, x, y):
        self.x = x
        self.y = y
        self.blocktype = blockType

        self.block = self.blocktypes[blockType]

        self.static = False

        self.blockColor = self.blockColors[blockType]
    
    def absolutPositions(self):
      block = self.block
      if(not self.static):
        block = self.calcRotBlock(self.rotateN, self.block)
      return self.calcAbsolutPos(block)
    
    def calcAbsolutPos(self, block):
        positions = []
        for i in range(4):
            for j in range(4):
              if block[i][j] == 1:
                  positions.append((self.x+i, self.y+j))
        return positions

    def calcRotBlock(self, rot,block):
      rotatedBlock = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
      for i in range(0,4):
        for j in range(0, 4):
              x = i
              y = j
              if rot == 1:
                x = j
                y = 3-i
              elif rot == 2:
                x = 3-i
                y = 3-j
              elif rot == 3:
                x = 3-j
                y = i
              rotatedBlock[i][j] = block[x][y]
      return rotatedBlock
              
    def clearLocalLine(self,y):
        for i in range(4):
            for j in range(4):
                if self.y+j == y:
                    self.block[i][j] = 0

    def getColor(self):
        return self.blockColor

    def rotate(self):
        self.rotateN = self.calcNextRot()
    
    def calcNextRot(self):
      return (self.rotateN + 1) % 4

    def makeStatic(self):
      self.static = True
      self.block = self.calcRotBlock(self.rotateN, self.block)

class TetrisE(Env):

  def __init__(self) -> None:
    super(TetrisE,self).__init__()
    #left, right, down,nothing, rotate
    self.action_space = spaces.Discrete(5)
    self.observation_space = spaces.Box(low=0, high=1,shape=(20,10,1),dtype=np.uint8)
    
    self.doRender = False

    self.reset()

  def step(self,action):
    GameField = self.gameField()
    if action == 0:
        self.move(-1,0,GameField)
    elif action == 1:
        self.move(1,0,GameField)
    elif action == 2:
        self.move(0,1,GameField)
    elif action == 3:
        self.rotate(GameField)

    self.move(0,1,GameField)
    lines = self.clearLines(GameField)

    return self.GameFieldToObservation(GameField), self.calcReward(lines), self.lost(GameField), {}

  def calcReward(self, linewCleared):
    a = linewCleared*linewCleared*linewCleared *linewCleared
    return a +1

  def GameFieldToObservation(self, gameField):
    observation = np.zeros((20,10,1))
    for i in gameField:
      if(i[1] >= 0 and i[1] < 20 and i[0]>= 0 and i[0]<10):
        observation[i[1]][i[0]] = 1
    return observation

  def reset(self):
    self.blocks = []
    self.newBlock()
    if self.doRender:
      pygame.quit()
      pygame.display.init()
      self.screen = pygame.display.set_mode((300, 600))
      self.screen.fill((0, 0, 0))
      pygame.display.flip()

    return self.GameFieldToObservation(self.gameField())

  def render(self, mode='human', close=False):
    self.screen.fill((0, 0, 0))
    for i in self.blocks:
      for j in i.absolutPositions():
        x,y = j
        pygame.draw.rect(self.screen, i.getColor(), (x*30, y*30, 30, 30))

    pygame.display.flip()

  def newBlock(self):
      self.CurrentBlock = TetrisBlock(random.randint(0,6), 4, -1)
      self.blocks.append(self.CurrentBlock)

  def lost(self, gameField):
    for i in gameField:
      if i[1] == 0:
        return True 
    return False

  def stop(self):
    pygame.quit()
  
  def clearLines(self,gameField):
    lines = []
    for j in range(0,20):
      cleared = True
      for i in range(0,10):
        if (i,j) not in gameField:
          cleared = False
      if cleared:
        print("line cleared")
        lines.append(j)

    for i in lines:
      for j in self.blocks:
        j.clearLocalLine(i)

    for i in self.blocks:
      i.y += len(lines)

    return len(lines)

  def move(self,x,y,gameField):
    currentBlock = self.CurrentBlock
    absolutePositions = currentBlock.absolutPositions()

    for i in absolutePositions:
      newPos = (i[0]+x,i[1]+y)
      if (i[0],i[1]+1) in gameField:
        self.CurrentBlock.makeStatic()
        self.newBlock()
        return
      elif i[1]+y > 19 or i[1]+y < -1:
        self.CurrentBlock.makeStatic()
        self.newBlock()
        return
      elif i[0]+x > 9 or i[0]+x < 0 or newPos in gameField:
        return
    self.CurrentBlock.x += x
    self.CurrentBlock.y += y
  
  def allowRotate(self,gameField):
    currentBlock = self.CurrentBlock
    absolutePositions = currentBlock.calcAbsolutPos(currentBlock.calcRotBlock(currentBlock.calcNextRot(),currentBlock.blocktypes[currentBlock.blocktype]))
    for i in absolutePositions:
      if i[0] > 9 or i[0] < 0 or i in gameField or i[1] > 19:
        return False
    return True	

  def rotate(self,gameField):
    if(self.allowRotate(gameField)):
      self.CurrentBlock.rotate()

  def gameField(self):
    gf=[]
    for i in self.blocks:
      if(self.CurrentBlock != i):
        for j in i.absolutPositions():
          gf.append(j)
    return gf

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

model =  Sequential([
    Dense(1, activation='relu', input_shape=(1,20,10,1)),
    Dense(200, activation='relu'),
    Dense(600, activation='relu'),
    Dense(2000, activation='relu'),
    Dense(600, activation='relu'),
    Dense(400, activation='relu'),
    Dense(200, activation='relu'),
    Dense(100, activation='relu'),
    Dense(50, activation='relu'),
    Flatten(),
    Dense(5, activation='relu'),
])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 20, 10, 1)      2         
                                                                 
 dense_1 (Dense)             (None, 1, 20, 10, 200)    400       
                                                                 
 dense_2 (Dense)             (None, 1, 20, 10, 600)    120600    
                                                                 
 dense_3 (Dense)             (None, 1, 20, 10, 2000)   1202000   
                                                                 
 dense_4 (Dense)             (None, 1, 20, 10, 600)    1200600   
                                                                 
 dense_5 (Dense)             (None, 1, 20, 10, 400)    240400    
                                                                 
 dense_6 (Dense)             (None, 1, 20, 10, 200)    8

In [ ]:
tet = TetrisE()

memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=5, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
#dqn.load_weights('brains/GreedyV1/brain.h5f'.format("TetrisE"))

dqn.fit(tet, nb_steps=100000, visualize=False, verbose=10)

dqn.save_weights('brains/BoltzV1/brain.h5f'.format("TetrisE"), overwrite=True)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training for 500000 steps ...


/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/usr/local/lib/python3.8/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)

    116/500000: episode: 1, duration: 284.303s, episode steps: 116, steps per second:   0, episode reward: 116.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 2.095 [0.000, 4.000],  loss: 0.500000, mae: 0.200000, mean_q: 0.000000
    232/500000: episode: 2, duration: 302.960s, episode steps: 116, steps per second:   0, episode reward: 116.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 2.009 [0.000, 4.000],  loss: 0.500000, mae: 0.200000, mean_q: 0.000000
    355/500000: episode: 3, duration: 321.034s, episode steps: 123, steps per second:   0, episode reward: 123.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 1.862 [0.000, 4.000],  loss: 0.500000, mae: 0.200000, mean_q: 0.000000
    441/500000: episode: 4, duration: 224.740s, episode steps:  86, steps per second:   0, episode reward: 86.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 2.465 [0.000, 4.000],  loss: 0.500000, mae: 0.200000, mean_q: 0.000000
    564/500000: episode: 5, duration: 318.658s, e